In [32]:
#import sys
import numpy as np
import pandas as pd
import tensorflow as tf 
#!{sys.executable} -m pip install tensorflow

In [52]:
"""
    Based on the training data given, we are able to extract 7 attributes:
    1. x accelerometer measurement
    2. y accelerometer measurement
    3. z accelerometer measurement
    4. x gyroscope measurement
    5. y gyroscope measurement
    6. z gyroscope measurement
    7. time stamp for accelerometer and gyroscope measures
    
    We start by creating a dataframe using the csv files provided for readability.
    
    @param x_file: contains the xyz accelerometers and xyz gyroscope measures from the lower limb
    @param x_time_file: contain the time stamps for the accelerometer and gyroscope measures
    @return dataframe of attributes and label
"""
def create_dataframe_X(x_file, x_time_file):
    df1 = pd.read_csv(x_file, sep = ',', names = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])
    df2 = pd.read_csv(x_time_file, names = ['Time stamp'])
    frames = [df1, df2]
    result = pd.concat(frames, axis = 1)
    
def create_dataframe_Y(y_file, y_time_file):
    df1 = pd.read_csv(y_file, names = ['Label'])
    df2 = pd.read_csv(y_time_file, names = ['Time stamp'])
    frames = [df1, df2]
    result = pd.concat(frames, axis = 1)
    print(result)

In [53]:
create_dataframe_&('TrainingData/subject_001_01__x.csv', 'TrainingData/subject_001_01__x_time.csv')

          X_acc     Y_acc     Z_acc     X_gyr     Y_gyr     Z_gyr  Time stamp
0      4.435275  8.196063  2.974488  0.014215 -0.039157 -0.016744       0.000
1      4.186920  8.344455  2.908057  0.005771 -0.004480 -0.003345       0.025
2      4.544637  8.408659  2.890000  0.007967  0.022412  0.001159       0.050
3      4.849308  8.411614  2.900692  0.027778 -0.010670 -0.014223       0.075
4      4.509190  8.118649  2.847298  0.021577 -0.045498 -0.021111       0.100
...         ...       ...       ...       ...       ...       ...         ...
37885  1.927349  9.584698  0.940000 -0.001852  0.015182 -0.000370     947.125
37886  1.930000  9.562658  0.874684  0.003927  0.006205  0.000671     947.150
37887  1.906144  9.555361  0.866144 -0.001111 -0.005937 -0.000032     947.175
37888  1.900380  9.575190  0.865570 -0.004195 -0.004688 -0.007653     947.200
37889  1.813466  9.584614  0.914613  0.000381 -0.010000 -0.011841     947.225

[37890 rows x 7 columns]
